<h1><center>
    Implementing transformer architectures. <br/>
</center></h1>

Objective : Implement the feed-forward pass of the original transformer network using only numpy (without machine learning frameworks)


In [ ]:
import numpy as np
import math

# Test on a single array:
forward_pass_array = np.array([101, 400, 500, 600, 107, 102])

## Step 1.1
Implement the sinus/cosinus positional encoding and token embedding used in the original paper [Attention Is All You Need](https://arxiv.org/abs/1706.03762).  


In [ ]:
'''
Defining source vocabulary, target vocabulary tokens, and corresponding word embeddings (b/w 0 and 1)
'''

emb_size = 512 

src_vocab = np.array([101,102,103,104,105,106,107,108,109,110,400,500,600])
tgt_vocab = np.array([i for i in range(700,720)])

np.random.seed(42)
src_emb = np.random.rand(len(src_vocab),emb_size)
np.random.seed(43)
tgt_emb = np.random.rand(len(tgt_vocab),emb_size)

In [ ]:
'''
Input = max_seq_len=6, pos_encod_len (d_model) = 128
To do = check if pos_encod_len = 128 or 512.
      = check with some python package 
'''

def positional_encoding(pos_encod_len, max_seq_len=6):
    MAX_SEQ_LEN = max_seq_len # maximum length of a sentence
    d_model = pos_encod_len # word embedding (and positional encoding) dimensions
    PE = np.zeros((MAX_SEQ_LEN, d_model))

    for pos in range(MAX_SEQ_LEN):
        for i in range(d_model//2):
            theta = pos / (10000 ** ((2*i)/d_model))
            PE[pos, 2*i] = math.sin(theta)
            PE[pos, 2*i+1] = math.cos(theta)
    return PE

In [ ]:
'''
Function to get word embeddings from token_list and vocab. 
Input = token_array: Array of input tokens in text. 
        vocab: vocabulary for tokens 
        vocab_emb: Vocabulary embedding
'''

def generate_word_embedding(token_array, vocab, vocab_emb):
    token_ids = np.array([np.where(src_vocab==token)[0][0] for token in list(token_array)])
    emb_matrix = vocab_emb[token_ids,:]
    return emb_matrix

## Step 1.2
Implement a dense layer with the number of hidden units as an argument.

In [ ]:
'''
Simple NN Layer - For FFNN 
X : Input data matrix 
hidden_units : Neurons 
'''

def dense_layer(X, hidden_units):
    W = np.random.rand(X.shape[1], hidden_units)
    b = np.zeros((1, hidden_units))
    return np.dot(X, W) + b

## Step 1.3
Implement all activation function such that they are compatible with the dense layer.

In [ ]:
'''
Implement Sigmoid, TanH, Relu
'''

def relu(X):
    return np.maximum(0, X)

def sigmoid(X):
    return 1/(1 + np.exp(-X))

def tanh(X):
    return np.tanh(X)

## Step 1.4
Implement dropout.

In [ ]:
'''
Function to implement dropout. 

hidden_layer = Weight Matrix 
prob = probability to keep neurons (0.8 = 80% of neurons will be kept)

Output = Modified weights. 


Places dropout used - 
1. FFNN 
2. Residual Connection (before passing it to layernorm)
3. Self Attention
4. Encoder Decoder Attention
'''
def dropout(hidden_layer, prob=0.9):
    mask = np.random.binomial(1, prob, size=hidden_layer.shape) / prob
    out = hidden_layer * mask
    return out.reshape(hidden_layer.shape)

## Step 1.5
Implement the skip (residual) connections - Implemented after dropout

In [ ]:
'''
Residual Connection - Each sub-layer (self-attention, ffnn) in each encoder/decoder has a 
                      residual connection around it, and is followed by a layer-normalization step.
'''

'''
Input: X = matrix from previous layer to output (input to out)
       out = output from previous layer. number of columns in X should be divisible by number of columns in out.
       
Output: Modified matrix with dropout
'''
def residual_connection(X, out):
    return dropout(X + out)




## Step 1.6
Implement layer normalization - Implemented after dropout

In [ ]:
'''
Input = weight matrix

Output = modified weight matrix after dropout. 
'''

def layer_norm(X):
    eps = 1e-5
    mean = X.mean(axis=1)
    std = X.std(axis=1)
    denominator = np.sqrt(std**2 + eps)[:,None]
    numerator = X - mean[:,None]
    layer_norm_score = numerator/denominator
    return layer_norm_score

## Step 1.7
Implement the attention mechanism.

In [ ]:
'''
Function to get query, key and value matrices. 
Input = Embedding Matrix X : (seq_len*embedding_size), req_dim : required dimension of q,k,v. (generally 64) 
Output = q,k,v matrices : dim = seq_len*req_dim
'''

def calculate_qkv(X, req_dim=64):
    Wq = np.random.rand(X.shape[1],req_dim)
    Wk = np.random.rand(X.shape[1],req_dim)
    Wv = np.random.rand(X.shape[1],req_dim)
    
    query = np.matmul(X,Wq)
    key = np.matmul(X,Wk)
    value = np.matmul(X,Wv)
    
    return query, key, value

In [ ]:
'''
Function to fill mask - required for masked attention
'''
def fill_mask(in_array, mask_val):
    array_copy = in_array.copy()
    array_copy = np.tril(array_copy)
    array_copy[np.where(array_copy==0)] = mask_val
    return array_copy

In [ ]:
'''
Function to return softmax along rows. 
Numerically stable softmax

'''


def softmax(x):
    max_val = np.max(x,axis=1,keepdims=True) #returns max of each row and keeps same dims
    e_x = np.exp(x - max_val) #subtracts each row with its max value
    sum_val = np.sum(e_x,axis=1,keepdims=True) #returns sum of each row and keeps same dims
    f_x = e_x / sum_val 
    return f_x


In [ ]:
'''
Self Attention = softmax((Q*K_transpose)/(sqrt(d_k)))*V
Use Mask for self-attention in decoder

Input = Query Matrix (Q) , Key Matrix (K), Value Matrix (V)
Dimensison of Q,K,V = sequence_length (n) * d_k (64 in the paper, dependes on Wq, Wk, Wv) 
Output = Self Attention Matrix : row r = attention scores over all the words for word r. 
         Dimension of Output = sequence_length (n) * 64

q = n * 64 
k = n * 64
n = n * 64

qkt = n*n
softmax(qkt) * v = n*64
'''

def self_attention(query, key, value, mask=False):
    d_k = query.shape[1]
    scores = np.matmul(query, key.transpose())/np.sqrt(d_k)
    if mask:
        scores = fill_mask(scores,mask_val=-np.inf)
    scores_softmax = softmax(scores)
    out_scores = dropout(np.matmul(scores_softmax,value))
    return out_scores

In [ ]:
'''
Function to get multihead attention scores

Input = X: Word Embedding matrix (seq_len * embedding_size) or output from previous encoder 
        num_heads: heads required, 
        qkv_req_dim : required dimension (columns) of q,k,v 
        
Outputs = multihead attention scores. (Dimension: sequence_len*512)
'''

def multihead_attention(X, num_heads, qkv_req_dim, mask=False):
    ## initialize array to store attention scores calculated from each head
    z_scores = np.zeros(shape = (X.shape[0], num_heads*qkv_req_dim), dtype = 'float')
    ## Lists to store query, key, vector values 
    query_list = [] 
    key_list = [] 
    value_list = []
    for head in range(num_heads):
        q, k, v = calculate_qkv(X,qkv_req_dim)
        query_list.append(q)
        key_list.append(k)
        value_list.append(v)
        attn_scores = self_attention(q,k,v,mask)
        
        ## Copy scores to z_scores
        start_idx = head*qkv_req_dim
        end_idx = (head+1)*qkv_req_dim
        z_scores[:,start_idx:end_idx] = attn_scores
    
    ## Iniitialize weight matrix to multiply with z_scores 
    w_o = np.random.rand(qkv_req_dim*num_heads, X.shape[1])
    
    multi_attn_score = np.matmul(z_scores, w_o)
    return query_list, key_list, value_list, multi_attn_score

## Step 1.8
Implement the positonal feed-forward network.

In [ ]:
'''
FFNN - 
Input= X: Input Matrix 
       n_hidden_layers = Number of hidden layers 
       num_neuron_list = list of no. of neurons in each layer. 
       activation_fn = activation function - can be 'relu','sigmoid' or 'tanh'

1. Dense_layer
2. Dropout 
3. Activation function

Add dropout
Only need Forward Pass. 
Complete this
'''

def ffnn(X, n_hidden_layers=2, num_neuron_list=[512,512], activation_fn='relu'):
    wt_matrices = [X]
    for layer in range(n_hidden_layers):
        dense = dense_layer(wt_matrices[layer],num_neuron_list[layer])
        drop = dropout(dense)
        if activation_fn == 'relu':
            out = relu(drop)
        elif activation_fn == 'tanh':
            out = tanh(drop)
        else:
            out = sigmoid(drop)
        wt_matrices.append(out)
    return wt_matrices[-1]

## Step 1.9
Implement the encoder attention.

In [ ]:
'''
Function to implement Encoder Attention. Refers to multihead attention function

Input = X: Word Embedding matrix (seq_len * embedding_size) or output from previous encoder. 
        num_heads: Number of attention heads. 
        qkv_req_dim : Dimenstion of query, key, value matrices (only number of columns)

Output = Multi head attention_scores (without masks)
'''
def encoder_attention(X,num_heads=8,qkv_req_dim=64):
    q_list, k_list, v_list, attention_scores = multihead_attention(X,num_heads,qkv_req_dim)
    return q_list, k_list, v_list, attention_scores

## Step 1.10
Implement the encoder.

In [ ]:
'''
Function to create a single encoder. 

Input: word_emb: Word Embedding Matrix (Including Positional Encoding) or output from previous layer. 
       num_heads: Number of attention heads. 
       qkv_req_dim: Dimenstion of query, key, value matrices (only number of columns)

Output: Final Scores, Queries, Keys, Values used in Multihead attention, 
        Processed Word Embedding Matrix after passing through Encoder

Steps: 
1. Multihead Self-Attention Layer
2. Residual Connection and LayerNorm (Add and Normalize)
3. FFNN  
4. Residual Connection and LayerNorm (Add and Normalize)
'''

def encoder(word_emb, num_heads=8, qkv_req_dim=64):
    q_list, k_list, v_list, attn_scores = encoder_attention(word_emb, num_heads, qkv_req_dim)
    layer_norm_scores = layer_norm(residual_connection(word_emb,attn_scores))
    #print("Shape of layer_norm_scores: " + str(layer_norm_scores.shape))
    ffnn_scores = ffnn(layer_norm_scores)
    #print("Shape of ffnn_scores: " + str(ffnn_scores.shape))
    final_scores = layer_norm(residual_connection(ffnn_scores,layer_norm_scores))
    return q_list, k_list, v_list, final_scores

In [ ]:
'''
Function to create encoder stack. 6 in original implementation. 

Input: word_emb: Word Embedding Matrix (Including Positional Encoding)
       stack_size: Number of encoders 
       num_heads: Number of attention heads. 
       qkv_req_dim: Dimenstion of query, key, value matrices (only number of columns)
       
Output: Final_Scores, Queries, Keys, Values from all encoder layers. 
'''
def encoder_stack(word_emb, stack_size = 6, num_heads=8, qkv_req_dim=64):
    ## list to store encoder outputs. 
    encoder_op_list = []
    
    ## Output of first encoder 
    encoder_op1 = encoder(word_emb, num_heads, qkv_req_dim)
    
    encoder_op_list.append(encoder_op1)
    
    ## Pass final scores from previous encoders as input to next encoder. 
    for i in range(1,stack_size):
        encoder_op = encoder(encoder_op_list[i-1][-1], num_heads, qkv_req_dim)
        encoder_op_list.append(encoder_op)
    
    return encoder_op_list

## Step 1.11
Implement the decoder attention.

In [ ]:
'''
Input = X: Output Word Embedding matrix (max_seq_len_decoder * embedding_size), or scores from previous decoder.
        num_heads: heads required, 
        qkv_req_dim : required dimension (columns) of q,k,v 
        
Output = Multi head attention_scores (with masks) and decoder query list (to be used for enc-dec attention)
'''
def decoder_self_attention(X,num_heads=8,qkv_req_dim=64,mask=True):
    decoder_query_list, _, _, attention_scores = multihead_attention(X,num_heads,qkv_req_dim,mask=True)
    return decoder_query_list, attention_scores

## Step 1.12
Implement the encoder-decoder attention.

In [ ]:
'''
Input = X: embedding matrix  
        enc_scores: final scores from encoder layer. 
        dec_scores: scores coming after add and norm from same decoder. 

Output = Multihead Encoder-Decoder Attention scores. 
'''

def encoder_decoder_attention(X, enc_scores, dec_scores, num_heads=8, qkv_req_dim=64,mask=False):
    ## initialize array to store attention scores calculated from each head
    z_scores = np.zeros(shape = (X.shape[0], num_heads*qkv_req_dim), dtype = 'float')
    for head in range(num_heads):
        _, k_enc, v_enc = calculate_qkv(enc_scores)
        q_dec, _, _ = calculate_qkv(dec_scores)

        attn_scores = self_attention(q_dec,k_enc,v_enc)
        ## Copy scores to z_scores
        start_idx = head*qkv_req_dim
        end_idx = (head+1)*qkv_req_dim
        z_scores[:,start_idx:end_idx] = attn_scores
    
    ## Iniitialize weight matrix to multiply with z_scores 
    w_o = np.random.rand(qkv_req_dim*num_heads, X.shape[1])
    
    multi_attn_score = dropout(np.matmul(z_scores, w_o))

    return multi_attn_score


## Step 1.13
Implement the decoder.

In [ ]:
'''
Function to initialize input to decoder. 

Input= start_tok_emb: start_token_embedding
       max_len: max_length of tokens to be predicted by decoder. 
       emb_size: dimension of embedding vector.

Output= init_dec_emb: Initial embedding for decoder. 
'''

def init_dec_input(start_tok_emb,max_len,emb_size=512):
    ## To adjust for start_tok_emb
    init_dec_emb = np.random.rand(max_len+1,emb_size)
    init_dec_emb[0] = start_tok_emb
    
    return init_dec_emb

In [ ]:
'''
Function to create a single decoder. 

Input: word_emb: Word Embedding Matrix (Including Positional Encoding) or output from previous layer. 
       enc_scores : Final Scores from Encoder 
       num_heads: Number of attention heads. 
       qkv_req_dim: Dimenstion of query, key, value matrices (only number of columns)

Output: Processed Word Embedding Matrix after passing through Decoder


Steps:
1. Masked Multihead Self-Attention Layer
2. Residual Connection and LayerNorm (Add and Normalize)
3. Encoder - Decoder Attention 
4. Residual Connection and LayerNorm (Add and Normalize)
5. FFNN 
6. Residual Connection and LayerNorm (Add and Normalize)
'''

def decoder(word_emb, enc_scores, num_heads=8, qkv_req_dim=64):
    #print("Input shape: " + str(word_emb.shape))
    ## can remove dec_query_list
    dec_query_list, masked_attn_scores = decoder_self_attention(word_emb,num_heads,qkv_req_dim,mask=True)
    #print("masked attn scores shape: " + str(masked_attn_scores.shape))
    layer_norm_scores1 = layer_norm(residual_connection(word_emb,masked_attn_scores))
    #print("layer norm scores1 shape: " + str(layer_norm_scores1.shape))
    enc_dec_attn_scores = encoder_decoder_attention(word_emb, enc_scores, layer_norm_scores1, num_heads, qkv_req_dim, mask=False)
    #print("Encoder Dec Scores shape: " + str(enc_dec_attn_scores.shape))
    layer_norm_scores2 = layer_norm(residual_connection(layer_norm_scores1,enc_dec_attn_scores))
    #print("layer norm2 Scores shape: " + str(layer_norm_scores2.shape))
    ffnn_scores = ffnn(layer_norm_scores2)
    #print("FFNN Scores shape: " + str(ffnn_scores.shape))
    final_scores = layer_norm(residual_connection(layer_norm_scores2,ffnn_scores))
    #print("final scores shape: " + str(final_scores.shape))
    return final_scores 


In [ ]:
'''
Function to create decoder stack. 

Input = word_emb: Word Embedding Matrix (Including Positional Encoding)
        stack_size: Number of encoders 
        enc_scores: Final Scores from Encoder.
        num_heads: Number of attention heads. 
        qkv_req_dim: Dimenstion of query, key, value matrices (only number of columns)

Output = Scores from Final Decoder Layer
'''

def decoder_stack(word_emb, enc_scores, stack_size = 6, num_heads=8, qkv_req_dim=64):
    ## list to store encoder outputs. 
    decoder_op_list = []
    
    ## Output of first decoder 
    decoder_op1 = decoder(word_emb, enc_scores, num_heads, qkv_req_dim)
    
    decoder_op_list.append(decoder_op1)
    #print("In decoder stack: ")
    #print("Len decoder op1: " + str(len(decoder_op1)))
    
    ## Pass final scores from previous encoders as encoder to next encoder. 
    for i in range(1,stack_size):
        #print("In decoder stack: " + str(i))
        decoder_op = decoder(decoder_op_list[i-1], enc_scores, num_heads, qkv_req_dim)
        decoder_op_list.append(decoder_op)
    
    return decoder_op_list

## Step 1.14
Implement the transformer architecture.

In [ ]:
'''
Function to find closest embedding to decoder output. 

Input: candidate_vec = vector for finding closest embedding to. 
       target_matrix = matrix from which closest embedding needs to be calculated. 
       
Output: Index of closest embedding in target_matrix
'''

from numpy import dot
from numpy.linalg import norm

def closest_embedding(candidate_vec,target_matrix):
    cos_metric = []
    for i in range(target_matrix.shape[0]):
        cos_val = dot(candidate_vec, target_matrix[i])/(norm(candidate_vec)*norm(target_matrix[i]))
        cos_metric.append(cos_val)
    
    return np.argmin(cos_metric)

In [ ]:
'''
1. Token Embedding. 
2. Add Positional Encoding - Need to pass this to Encoder 
3. Encoder 1st part - Multi-head (8) Self Attention  - How to make parallize the inputs? (do we need threading?)
4. Encoder 2nd part - LayerNorm (Residual Connection + O/p from Multihead self-attention)
5. Encoder 3rd part - Pass output from 4 to FFNN. 
6. Repeat Residual Connection + O/p from Multihead self-attention. 
7. Make 6 blocks of Encoder and repeat 3-6. 
8. The output of 7 will be passed to each decoder. The Key and Vector matrix are fed to decoder (Need to check what goes exactly into decoder)
9. Decoder 1st part - Multi-head (8) Masked Self Attention
10. Decoder 2nd part - LayerNorm (Residual Connection + O/p from Multihead masked self-attention)
11. Decoder 3rd part - Encoder - Decoder Attention
12. Decoder 4th part - LayerNorm (Residual Connection + O/p from Enc-Dec self-attention)
13. Decoder 5th part - Pass output from 12 to FFNN. 
14. Repeat Residual Connection + O/p from 13.
15. 6 blocks of Decoder. 
16. Predict 1st word. 
17. Pass 16 as Decoder Input and repeat 9-16 for predicting subsequent words. 
18. Check how to add positional encoding to output of decoder.
'''




'\n1. Token Embedding. \n2. Add Positional Encoding - Need to pass this to Encoder \n3. Encoder 1st part - Multi-head (8) Self Attention  - How to make parallize the inputs? (do we need threading?)\n4. Encoder 2nd part - LayerNorm (Residual Connection + O/p from Multihead self-attention)\n5. Encoder 3rd part - Pass output from 4 to FFNN. \n6. Repeat Residual Connection + O/p from Multihead self-attention. \n7. Make 6 blocks of Encoder and repeat 3-6. \n8. The output of 7 will be passed to each decoder. The Key and Vector matrix are fed to decoder (Need to check what goes exactly into decoder)\n9. Decoder 1st part - Multi-head (8) Masked Self Attention\n10. Decoder 2nd part - LayerNorm (Residual Connection + O/p from Multihead masked self-attention)\n11. Decoder 3rd part - Encoder - Decoder Attention\n12. Decoder 4th part - LayerNorm (Residual Connection + O/p from Enc-Dec self-attention)\n13. Decoder 5th part - Pass output from 12 to FFNN. \n14. Repeat Residual Connection + O/p from 13

In [ ]:

'''
Function to execute transformer architecture

Input= forward_pass_array: input token array
       src_vocab: source vocabulary 
       src_emb: source embedding
       dec_max_len: decoding time steps to stop prediction.  
       end_token: end token for decoder. 
'''

def transformer(forward_pass_array, src_vocab=src_vocab, src_emb=src_emb, dec_max_len=10, end_token=715):
    ## Get token embeddings. 

    ## input_token_emb Dimension = len(forward_pass_array)*512 
    input_token_emb = generate_word_embedding(token_array=forward_pass_array, vocab=src_vocab, vocab_emb=src_emb)
    input_seq_len = len(forward_pass_array)


    ## Positional Encoding. Dimension = input_seq_len*512
    enc_in_pos = positional_encoding(pos_encod_len=512, max_seq_len=input_seq_len)


    ## Add token embedding and positional encoding. Dimension = input_seq_len*512
    word_emb = input_token_emb + enc_in_pos


    ## Encoder Blocks 
    encoder_output = encoder_stack(word_emb)


    ## Output from last Encoder.
    last_encoder_op = encoder_output[-1]
    enc_query, enc_key, enc_value, enc_scores = last_encoder_op



    ## Decoder Termination Criteria = 10 output words. 
    #dec_max_len = 10 

    ## Intitialize start_tok_emb with dimension 512. 
    start_tok_emb = np.random.rand(512)

    ## Decoder input token embedding
    dec_in_token = init_dec_input(start_tok_emb,max_len=dec_max_len)

    ## Positional Encoding 
    dec_in_pos = positional_encoding(pos_encod_len=512, max_seq_len=dec_max_len+1)

    ## Add token embedding and positional encoding
    dec_in = dec_in_token+dec_in_pos

    predictions = []
    for i in range(1,dec_max_len+1):
        ## Decoder Blocks 
        decoder_output = decoder_stack(dec_in,enc_scores)

        ## Output from last decoder. 
        dec_scores = decoder_output[-1]

        ## Pass output to linear layer 
        linear_layer_scores = ffnn(dec_scores,n_hidden_layers=1,num_neuron_list=[512])

        ## Pass linear_layers to softmax
        softmax_scores = softmax(linear_layer_scores)

        ## Check closest token embedding and token to softmax_scores at ith time step.
        pred_token_arg = closest_embedding(softmax_scores[i], tgt_emb)

        pred_token = tgt_vocab[pred_token_arg]

        predictions.append(pred_token)
        dec_in[i] = tgt_emb[pred_token_arg]

        ## Add positional encodings in each time step.
        dec_in += positional_encoding(pos_encod_len=512, max_seq_len=dec_max_len+1)

        if pred_token==end_token:
          break
        
    return predictions

## Step 1.15
Test the forward pass using the follow array.

In [ ]:
forward_pass_array = np.array([101, 400, 500, 600, 107, 102])
output_predictions = transformer(forward_pass_array)
print(output_predictions)

[706, 700, 711, 707, 708, 702, 716, 715]


In [ ]:
'''
Testing with 20 random sequences. 
'''

test_seq = [] 
for i in range(20):
    arr = np.random.choice(src_vocab, size = (np.random.randint(low=1, high=len(src_vocab)-1)))
    test_seq.append(arr)

pred_test_seq = []

for i in range(len(test_seq)):
    transformer_input = test_seq[i]
    pred_test_seq.append(transformer(transformer_input))
    print("Input " + str(i) + ": " + str(test_seq[i]))
    print("Output " + str(i) + ": " + str(pred_test_seq[i]))
    print("\n")

Input 0: [104 105 110 400 103 107 400]
Output 0: [711, 700, 700, 705, 709, 713, 717, 719, 702, 710]


Input 1: [400]
Output 1: [718, 704, 709, 711, 719, 709, 710, 704, 702, 709]


Input 2: [101 101 110 101 600 600]
Output 2: [706, 708, 719, 711, 712, 711, 702, 714, 711, 707]


Input 3: [101 104]
Output 3: [714, 704, 716, 706, 700, 708, 715]


Input 4: [108 105 107 105 110 108 107]
Output 4: [710, 717, 713, 700, 706, 717, 700, 700, 705, 700]


Input 5: [105 105 500]
Output 5: [701, 710, 709, 705, 711, 700, 706, 707, 707, 703]


Input 6: [103 105 400]
Output 6: [704, 700, 713, 716, 714, 711, 700, 713, 701, 717]


Input 7: [109 109 106 109 400 103]
Output 7: [715]


Input 8: [105 108 108 104 104 107 102]
Output 8: [703, 705, 707, 713, 705, 700, 705, 708, 701, 716]


Input 9: [103]
Output 9: [716, 715]


Input 10: [104 101 400 108 108 103 101 400 102]
Output 10: [707, 715]


Input 11: [105 108 108 400 106 108 110 102 109 105]
Output 11: [715]


Input 12: [110 104 108 600 109 104]
Output 12